# <center>LLM - Detect AI Generated Text</center>

This competition challenges participants to develop a machine learning model that can accurately detect **whether an essay was written by a student or an LLM**. The competition dataset comprises a mix of student-written essays and essays generated by a variety of LLMs.

Team Members: 毛柏毅, 朱誼學, 許木羽, 張立誠

## Configuration

In [1]:
import transformers as T
from datasets import Dataset
import torch
# from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
import torch.nn.functional as F
import kagglehub
import numpy as np

import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display, HTML

In [2]:
from typing import Literal

HOST: Literal['Localhost', 'Interactive', 'Batch'] = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost')
IS_RERUN: bool = os.getenv('KAGGLE_IS_COMPETITION_RERUN')

print(f'HOST: {HOST}, IS_RERUN: {IS_RERUN}')

HOST: Localhost, IS_RERUN: None


In [3]:
device = torch.device(
    ("cuda:0" if torch.cuda.is_available()
     else "mps" if torch.backends.mps.is_available()
     else "cpu"))
print(device)

cuda:0


## Data

### Load Data

In [4]:
def get_kaggle_csv(dataset: str, name: str, is_comp: bool = False) -> pd.DataFrame:
    assert name.endswith('.csv')
    if IS_RERUN:
        return pd.read_csv(f'/kaggle/input/{dataset}/{name}')
    if is_comp:
        path = kagglehub.competition_download(dataset)
    else:
        path = kagglehub.dataset_download(dataset)
    return pd.read_csv(Path(path) / name)

In [5]:
if IS_RERUN:
    df_train = get_kaggle_csv('daigt-datamix', 'train_essays.csv')
    df_test = get_kaggle_csv('llm-detect-ai-generated-text', 'test_essays.csv', is_comp=True)
else:
    df = get_kaggle_csv('dogeon188/daigt-datamix', 'train_essays.csv')
    # split df_train into train and test
    df = df.sample(frac=1).reset_index(drop=True)
    df_test = df.iloc[-1000:].copy()
    df_train = df.iloc[:5000].copy()
    # Up sampling -> used to balance the number of data (generated = 0 or 1)
    human = df[df['generated']==0].copy().sample(frac=1).reset_index(drop=True)
    minority = df_train[df_train['generated']==0].shape[0]
    majority = df_train[df_train['generated']==1].shape[0]
    up_sampling = human[:majority-minority]
    df_train = pd.concat((df_train, up_sampling)).sample(frac=1).reset_index(drop=True)
    assert df_train[df_train['generated']==0].shape[0] == df_train[df_train['generated']==1].shape[0]
    print(df_train.shape)

(8744, 4)


## Model

In [6]:
tokenizer = T.AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=False)
tokenizer.save_pretrained("./src_for_comp/tokenizer")

# tokenizer = T.AutoTokenizer.from_pretrained("src/tokenizer")

# Define the hyperparameters
lr = 3e-5
epochs = 5
train_batch_size = 16
validation_batch_size = 16
test_batch_size = 16

### Preprocess Data

In [7]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation", "test"]
        if split != 'test':
            self.data = df[split]
        else:
            self.data = df
    def __getitem__(self, index):
        d = self.data.iloc[index]
        return d

    def __len__(self):
        return len(self.data)
    
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    generated = [item['generated'] for item in batch]

    encoded_inputs = tokenizer(
        texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    )

    generated_tensor = torch.tensor(generated)

    return {
        'input_ids': encoded_inputs['input_ids'],
        'token_type_ids': encoded_inputs['token_type_ids'],
        'attention_mask': encoded_inputs['attention_mask'],
        'generated': generated_tensor
    }

split_ratio = 0.85
split_idx = int(len(df_train) * split_ratio)
df_split = {"train": df_train[:split_idx], "validation": df_train[split_idx:]}


ds_train = CustomDataset(df_split, "train")
ds_validation = CustomDataset(df_split, "validation")
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=train_batch_size, collate_fn=collate_fn)
dl_validation = torch.utils.data.DataLoader(ds_validation, batch_size=validation_batch_size, collate_fn=collate_fn)

### Model

In [8]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.cpu().numpy()
    auc_score = roc_auc_score(labels, logits)

    return auc_score

class Deberta(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.deberta = T.AutoModel.from_pretrained("microsoft/deberta-v3-base", num_labels=1)
        self.deberta.save_pretrained("./src_for_comp/deberta")
        self.deberta.gradient_checkpointing_enable()

        self.mlp_head = torch.nn.Sequential(
            torch.nn.Linear(768, 384),
            torch.nn.ReLU(),
            # torch.nn.Dropout(p=0.2),
            torch.nn.Linear(384, 192),
            torch.nn.ReLU(),
            torch.nn.Linear(192, 1)
        )

        self.activation = torch.nn.Sigmoid()
    def forward(self, **kwargs):
        input_ids = kwargs['input_ids']
        attention_mask = kwargs['attention_mask']
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        # Mean pooling
        # reshape attention_mask, used to filter the valid tokens
        mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()  # [batch_size, seq_len, hidden_size]
        # summation for valid tokens
        sum_embeddings = torch.sum(last_hidden_state * mask, dim=1)  # [batch_size, hidden_size]
        # num of valid tokens
        sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
        x = sum_embeddings / sum_mask
        
        x = self.mlp_head(x)
        x = self.activation(x)
        
        return {
            'generated': x
        }


In [9]:
model = Deberta().to(device)
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.01)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
loss_fn = torch.nn.MSELoss()

In [10]:
for ep in range(epochs):
    pbar = tqdm(dl_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    model.train()

    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        generated = batch['generated'].float().to(device)
        
        optimizer.zero_grad()
        pred = model(input_ids = input_ids, attention_mask = attention_mask)
        loss = loss_fn(pred['generated'].squeeze(-1), generated)
        loss.backward()
        optimizer.step()

        pred_gen = list(map(lambda x: x.item(), pred['generated']))
        pbar.set_postfix({"Loss": f"{loss:.4f}", "ROC AUC": f"{compute_metrics((pred_gen, generated)):.4f}"})
        

    pbar = tqdm(dl_validation)
    pbar.set_description(f"Validation epoch [{ep+1}/{epochs}]")
    model.eval()

    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            generated = batch['generated'].to(device)
            pred = model(input_ids=input_ids, attention_mask=attention_mask)

            # Scoring
            pred_gen = list(map(lambda x: x.item(), pred['generated']))
            pbar.set_postfix({"ROC AUC": f"{compute_metrics((pred_gen, generated)):.4f}"})


Validation epoch [5/5]: 100%|██████████| 82/82 [00:18<00:00,  4.52it/s, ROC AUC=0.9844]


## Save Model

In [11]:
torch.save(model.state_dict(), "./src_for_comp/model")

## Prediction

In [12]:
model = Deberta().to(device)
model.load_state_dict(torch.load("./src_for_comp/model", weights_only=True))

ds_test = CustomDataset(df_test.copy(), "test")
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=test_batch_size, collate_fn=collate_fn)

pbar = tqdm(dl_test)
pbar.set_description(f"Test")
model.eval()

final_preds = []

with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            generated = batch['generated'].to(device)
            
            pred = model(input_ids=input_ids, attention_mask=attention_mask)
            pred_gen = list(map(lambda x: x.item(), pred['generated']))

            final_preds.extend(pred_gen)

# final_preds = ...  # should be a 1D array of predictions, with the same length as df_test, and values in [0, 1]

Test: 100%|██████████| 63/63 [00:13<00:00,  4.50it/s]


## Evaluation

In [13]:
# validation
if not IS_RERUN:
    from sklearn.metrics import roc_auc_score

    auc_score = roc_auc_score(df_test['generated'], final_preds)
    
    print(f"ROC AUC: {auc_score:.4f}")

ROC AUC: 0.9192


## Submission

In [14]:
df_test['generated'] = final_preds
submission = df_test[['id' if IS_RERUN else 'prompt_id', 'generated']]
submission.to_csv('submission.csv', index=False)